In [1]:
import pandas as pd
import geopandas as gpd
import pyogrio

In [2]:
df = pd.read_csv("C:/Users/eleonore.kong/Documents/InSAR/ccr_arretes.csv")
df = df[df['Péril'] == 'Sécheresse']
# df = df[df['Décision'] == 'Reconnue']
df['annee_debut_reco'] = df['Début reconnaissance'].str[-4:].astype(int)
df['annee_fin_reco'] = df['Fin reconnaissance'].str[-4:].astype(int)
df = df[(df['annee_debut_reco'] > 2015) & (df['annee_debut_reco'] < 2022)]
df

,code NOR,Arrêté lien,Arrêté numéro,Insee,Commune,Début reconnaissance,Fin reconnaissance,Arrêté du,Parution au JO le,Péril,Franchise,Décision,annee_debut_reco,annee_fin_reco
1099,IOME2330891A,https://catastrophes-naturelles.ccr.fr/detail-...,834,21535,RUFFEY LES ECHIREY,30/09/2021,31/12/2021,21/11/2023,10/12/2023,Sécheresse,-,Non reconnue,2021,2021
1698,IOME2324730A,https://catastrophes-naturelles.ccr.fr/detail-...,829,14059,BENERVILLE SUR MER,01/07/2021,31/12/2021,19/09/2023,20/10/2023,Sécheresse,-,Non reconnue,2021,2021
1708,IOME2324730A,https://catastrophes-naturelles.ccr.fr/detail-...,829,16015,ANGOULEME,01/04/2016,30/09/2016,19/09/2023,20/10/2023,Sécheresse,-,Non reconnue,2016,2016
1710,IOME2324730A,https://catastrophes-naturelles.ccr.fr/detail-...,829,16120,DIRAC,01/01/2016,06/12/2016,19/09/2023,20/10/2023,Sécheresse,-,Non reconnue,2016,2016
1730,IOME2324730A,https://catastrophes-naturelles.ccr.fr/detail-...,829,27216,EMALLEVILLE,01/01/2020,03/11/2020,19/09/2023,20/10/2023,Sécheresse,-,Non reconnue,2020,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11805,INTE1727359A,https://catastrophes-naturelles.ccr.fr/detail-...,662,85266,SAINT PROUANT,01/04/2016,30/09/2016,24/10/2017,01/11/2017,Sécheresse,-,Non reconnue,2016,2016
11806,INTE1727359A,https://catastrophes-naturelles.ccr.fr/detail-...,662,86207,LA ROCHE POSAY,01/01/2016,31/03/2016,24/10/2017,01/11/2017,Sécheresse,Simple,Reconnue,2016,2016
11807,INTE1727359A,https://catastrophes-naturelles.ccr.fr/detail-...,662,88206,GIRONCOURT SUR VRAINE,01/04/2016,31/12/2016,24/10/2017,01/11/2017,Sécheresse,-,Non reconnue,2016,2016
11808,INTE1727359A,https://catastrophes-naturelles.ccr.fr/detail-...,662,89316,PROVENCY,01/07/2016,30/09/2016,24/10/2017,01/11/2017,Sécheresse,-,Non reconnue,2016,2016


In [3]:
data = pd.DataFrame(df.groupby(['Insee', 'Décision']).size()).reset_index()
data = data.rename(columns={0: 'Count'})
data = data.pivot_table(index=['Insee'], columns='Décision', values='Count', fill_value=0)

data.reset_index(inplace=True)
data.reset_index(drop=True, inplace=True)

In [4]:
gdf = pyogrio.read_dataframe("C:/Users/eleonore.kong/Documents/InSAR/communes.geojson")
joined = gdf.merge(data, left_on='insee_com', right_on='Insee', how='left')

In [5]:
pyogrio.write_dataframe(joined,"C:/Users/eleonore.kong/Documents/InSAR/communes_catnat_secheresse.gpkg", 'communes_2016-2021')